<a href="https://colab.research.google.com/github/ganeshagrahari/Pytorch-Framework/blob/main/14_LSTM_using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install nltk

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [39]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""


In [40]:
# Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [41]:
#tokenize
tokens = word_tokenize(document.lower())

In [42]:
#build vocab
vocab = {'<unk>' : 0}

for token in Counter(tokens).keys():
  if token not in vocab:
    vocab[token] = len(vocab)

len(vocab)

289

In [43]:
#extract sentences from data
input_sentences = document.split('\n')

In [44]:
def text_to_indices(sentence,vocab):
  numerical_sentence = []
  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence



In [45]:
input_numrical_sentences = []
for sentence in input_sentences:
  input_numrical_sentences.append(text_to_indices(word_tokenize(sentence.lower()),vocab))


In [46]:
len(input_numrical_sentences)

78

In [47]:
training_sequences = []
for sentence in input_numrical_sentences:
  for i in range(1,len(sentence)):
    training_sequences.append(sentence[:i+1])


In [48]:
len(training_sequences)

942

In [49]:
training_sequences[:5]

[[1, 2], [1, 2, 3], [4, 5], [4, 5, 2], [4, 5, 2, 6]]

In [50]:
len_list = []
for sequence in training_sequences:
  len_list.append(len(sequence))

max(len_list)


62

In [51]:
training_sequences[0]

[1, 2]

In [52]:
padded_tarining_sequences = []
for sequence in training_sequences:
  padded_tarining_sequences.append([0]*(max(len_list)-len(sequence))+sequence)


In [53]:
len(padded_tarining_sequences[0])

62

In [54]:
padded_tarining_sequences = torch.tensor(padded_tarining_sequences,dtype = torch.long)

In [55]:
padded_tarining_sequences.shape

torch.Size([942, 62])

In [56]:
x = padded_tarining_sequences[:,:-1]
y = padded_tarining_sequences[:,-1]

In [57]:
x

tensor([[  0,   0,   0,  ...,   0,   0,   1],
        [  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   0,   0,   4],
        ...,
        [  0,   0,   0,  ...,   0, 285, 176],
        [  0,   0,   0,  ..., 285, 176, 286],
        [  0,   0,   0,  ..., 176, 286, 287]])

In [58]:
y

tensor([  2,   3,   5,   2,   6,   7,   8,   9,  10,  11,   3,  12,  13,  14,
         15,   6,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  18,  26,
         27,  28,  29,  30,   5,   2,  31,  32,  27,   2,   6,  33,  31,  32,
         27,   2,   6,   5,  34,  35,  30,  36,   2,  31,   6,   7,  37,  38,
         39,  34,  40,  28,  41,  12,  42,  30,  15,   5,   2,  43,  27,   2,
         11,   3,  33,  45,  46,  47,   2,  48,  49,  50,  52,  53,   8,   9,
         10,  54,   8,   9,  10,   8,  57,  58,  60,  59,  64,  65,  66,   2,
         67,  43,  68,  69,  70,  50,  71,  72,  58,  73,  74,  46,  17,  75,
         27,  76,   3,  33,  78,  74,  73,  72,  58,  79,  80,  81,  17,  75,
         27,  76,   3,  82,  83,  84,  30,  85,  86,  87,  17,  88,  89,  33,
         45,  86,  90,  17,  91,  27,   2,  89,  33,  93,  94,  95,  80,  96,
         78,  36,  97,  85,  22,  87,  17,  89,  22,  65,  98,  99,  73, 100,
          2,  91,  30,  65,  86, 101,   2, 102, 103,  33,  76, 1

In [59]:
class CustomDataset(Dataset):

  def __init__(self,x,y):
    self.x = x
    self.y = y


  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self,index):
    return self.x[index],self.y[index]

In [60]:
dataset = CustomDataset(x,y)

In [61]:
dataset[0]

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 tensor(2))

In [62]:
dataloader = DataLoader(dataset,batch_size=32,shuffle=True)

In [63]:
for input, output in dataloader:
  print(input,output)

tensor([[  0,   0,   0,  ..., 151, 172, 173],
        [  0,   0,   0,  ...,  22,  23,  24],
        [  0,   0,   0,  ..., 223, 186,  78],
        ...,
        [  0,   0,   0,  ...,   4, 263, 264],
        [  0,   0,   0,  ..., 114,  78,  93],
        [  0,   0,   0,  ...,  44,  45, 157]]) tensor([174,  25,  22, 228, 206,   2,  22,   2,  33,  45,  59, 176, 260,  68,
        278, 170, 264,  24, 140,  23,  39,  26, 144,  22,  33,   8, 282, 164,
         78, 252,   2,  22])
tensor([[  0,   0,   0,  ...,  30,  36,   2],
        [  0,   0,   0,  ...,  86, 136, 205],
        [  0,   0,   0,  ...,   2, 143,  78],
        ...,
        [  0,   0,   0,  ...,  23,  24,  25],
        [  0,   0,   0,  ..., 176,   2, 175],
        [  0,   0,   0,  ...,  24, 257,  93]]) tensor([ 31, 206,  45,  65, 252, 219,  24,  86,   5,  45, 156,  81,  76,  23,
         23,  93,  17, 176,  75,   6,  27,   8,  65,  95, 142,   2, 142, 198,
          8,  18,  78,   2])
tensor([[  0,   0,   0,  ...,  89, 132,  22],
    

In [94]:
  class LSTMModel(nn.Module):
    def __init__(self,vocab_size):
      super().__init__()
      self.embedding = nn.Embedding(vocab_size,100)
      self.lstm = nn.LSTM(100,150,batch_first=True)
      self.fc = nn.Linear(150,vocab_size)

    def forward(self,x):
      embedded = self.embedding(x)
      intermediate_hidden_state,(final_hidden_state,final_cell_state)=self.lstm(embedded)
      output = self.fc(final_hidden_state.squeeze(0))

      return output

In [95]:
x = nn.Embedding(289,100)
y = nn.LSTM(100,150,batch_first=True)


In [96]:
a = dataset[0][0].unsqueeze(0)

In [97]:
b = x(a)

In [98]:
c ,d =y(b)

In [99]:
c.shape #set of all intermediate hidden states

torch.Size([1, 61, 150])

In [100]:
e, f = d

In [101]:
e.shape

torch.Size([1, 1, 150])

In [102]:
f.shape

torch.Size([1, 1, 150])

In [103]:
z = nn.Linear(150,289)

In [104]:
z(f.squeeze(0)).shape

torch.Size([1, 289])

In [105]:
model = LSTMModel(len(vocab))

In [106]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [107]:
model.to(device)

LSTMModel(
  (embedding): Embedding(289, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=289, bias=True)
)

In [108]:
epochs = 50
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [109]:
#training loop
for epoch in range(epochs):
  total_loss = 0
  for batch_x,batch_y in dataloader:
    batch_x,batch_y =batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    output = model(batch_x)

    loss = criterion(output, batch_y)

    loss.backward()

    optimizer.step()

    total_loss+=loss.item()
  print(f"Epoch: {epoch+1},loss:{total_loss:.4f}")

Epoch: 1,loss:165.6534
Epoch: 2,loss:145.8427
Epoch: 3,loss:134.1171
Epoch: 4,loss:121.7248
Epoch: 5,loss:109.1317
Epoch: 6,loss:97.4898
Epoch: 7,loss:87.1761
Epoch: 8,loss:77.5032
Epoch: 9,loss:68.2854
Epoch: 10,loss:60.4501
Epoch: 11,loss:53.1784
Epoch: 12,loss:46.5024
Epoch: 13,loss:40.6929
Epoch: 14,loss:35.8724
Epoch: 15,loss:31.0778
Epoch: 16,loss:27.4202
Epoch: 17,loss:24.4376
Epoch: 18,loss:21.7098
Epoch: 19,loss:19.4118
Epoch: 20,loss:17.3118
Epoch: 21,loss:15.3201
Epoch: 22,loss:13.9379
Epoch: 23,loss:12.7019
Epoch: 24,loss:11.7668
Epoch: 25,loss:10.8319
Epoch: 26,loss:9.9440
Epoch: 27,loss:9.2920
Epoch: 28,loss:8.8350
Epoch: 29,loss:8.3441
Epoch: 30,loss:7.8248
Epoch: 31,loss:7.5162
Epoch: 32,loss:7.1366
Epoch: 33,loss:6.7617
Epoch: 34,loss:6.5594
Epoch: 35,loss:6.3391
Epoch: 36,loss:5.9353
Epoch: 37,loss:5.7234
Epoch: 38,loss:5.5930
Epoch: 39,loss:5.4312
Epoch: 40,loss:5.2449
Epoch: 41,loss:5.1246
Epoch: 42,loss:5.0372
Epoch: 43,loss:4.9627
Epoch: 44,loss:4.8309
Epoch: 45,l

In [ ]:
#prediction
def prediction(model,vocab,text):
  #tokenize
  tokenized_text = word_tokenize(text.lower())

  #text -> numerical indices

  #padding

  #send to model

  #predicted index

  #merge with text